In [1]:
import pandas as pd
df = pd.read_csv("Wind_2022_2024_raw.csv")
df.describe


<bound method NDFrame.describe of                    time  temperature_2m  apparent_temperature  wind_speed_10m  \
0      2022-01-01T00:00             4.3                   1.7             7.9   
1      2022-01-01T01:00             3.8                   0.9             8.3   
2      2022-01-01T02:00             3.5                   0.6             8.4   
3      2022-01-01T03:00             3.0                   0.1             8.0   
4      2022-01-01T04:00             1.9                  -0.9             6.4   
...                 ...             ...                   ...             ...   
26299  2024-12-31T19:00             1.0                  -1.2             0.6   
26300  2024-12-31T20:00             0.1                  -2.3             1.3   
26301  2024-12-31T21:00            -0.4                  -3.0             1.9   
26302  2024-12-31T22:00            -0.4                  -3.0             1.6   
26303  2024-12-31T23:00            -0.5                  -3.2             2

In [3]:
import pandas as pd

fn = "AGGREGATED_GENERATION_PER_TYPE_GENERATION_1hour_2022.csv"
outputfile = "gen_hourly_2022.csv"
gcol = "Generation (MW)"

df = pd.read_csv(fn,low_memory=False)

# 1) Parse MTU start/end (handles " (CET)" / " (CEST)" suffixes)
mtu = df["MTU (CET/CEST)"].str.extract(
    r'(?P<start>\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2})\s*(?:\((?:CET|CEST)\))?\s*-\s*'
    r'(?P<end>\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2})\s*(?:\((?:CET|CEST)\))?'
)
df["Start"] = pd.to_datetime(mtu["start"], dayfirst=True, errors="coerce")
df["End"]   = pd.to_datetime(mtu["end"],   dayfirst=True, errors="coerce")

# 2) Make generation numeric (also handle decimal commas)
df[gcol] = pd.to_numeric(df[gcol].astype(str).str.replace(",", "."), errors="coerce")

# 3) Keep only good rows, set index for resampling
df = df.dropna(subset=["Start", gcol]).set_index("Start").sort_index()

# 4) Grouping columns if present
group_cols = [c for c in ["Area", "Production Type"] if c in df.columns]

# === Hourly average power (MW) ===
hourly_avg = (
    df.groupby(group_cols + [pd.Grouper(freq="1h")])[gcol]
      .mean()
      .reset_index()
      .rename(columns={gcol: "Generation (MW)", "Start": "Hour"})
)

# === Hourly energy (MWh) from 15-min MW readings (each 15-min = 0.25 h) ===
hourly_mwh = (
    df.assign(EnergyMWh=df[gcol] * 0.25)
      .groupby(group_cols + [pd.Grouper(freq="1h")])["EnergyMWh"]
      .sum()
      .reset_index()
      .rename(columns={"Start": "Hour", "EnergyMWh": "Energy (MWh)"})
)
# Move 'Hour' to first and rename to 'datetime'
for df_out in [hourly_avg, hourly_mwh]:
    if "Hour" in df_out.columns:
        df_out.rename(columns={"Hour": "datetime"}, inplace=True)
        # Move to first column
        cols = ["datetime"] + [c for c in df_out.columns if c != "datetime"]
        df_out = df_out[cols]
        # assign back
        if "Generation (MW)" in df_out.columns:
            hourly_avg = df_out
        else:
            hourly_mwh = df_out

# 5) Save (optional)
#hourly_avg.to_csv("generation_hourly_average_MW.csv", index=False)
hourly_mwh.to_csv(outputfile, index=False)

#print(hourly_avg.head())
print(hourly_mwh.head())

             datetime    Area Production Type  Energy (MWh)
0 2022-01-01 00:00:00  BZN|ES         Biomass        131.50
1 2022-01-01 01:00:00  BZN|ES         Biomass        133.25
2 2022-01-01 02:00:00  BZN|ES         Biomass        131.00
3 2022-01-01 03:00:00  BZN|ES         Biomass        133.00
4 2022-01-01 04:00:00  BZN|ES         Biomass        132.00
